In [1]:
import os 
import litellm
from typing import Final
from dotenv import load_dotenv
from textwrap import dedent, wrap
from pydantic import BaseModel, Field

load_dotenv(override=True)



num_dimention_instances: Final[int] = 50


In [3]:
MODEL_NAME: Final[str] = os.environ["MODEL_NAME"]
MODEL_NAME

'gemini/gemini-2.5-flash'

In [4]:
from litellm import get_supported_openai_params

params = get_supported_openai_params(model=MODEL_NAME)

custom_llm_provider = litellm.get_llm_provider(model=MODEL_NAME)

print(custom_llm_provider)

assert "response_format" in params

print(params)

from litellm import supports_response_schema

assert supports_response_schema(model=MODEL_NAME)

('gemini-2.5-flash', 'gemini', None, None)
['temperature', 'top_p', 'max_tokens', 'max_completion_tokens', 'stream', 'tools', 'tool_choice', 'functions', 'response_format', 'n', 'stop', 'logprobs', 'frequency_penalty', 'modalities', 'parallel_tool_calls', 'web_search_options', 'reasoning_effort', 'thinking']


In [7]:
def generate_dimention_list(dimention_list: str):
    SYSTEM_PROMPT = dedent(f"""
        You are AI assistant that helps generate instances for dimentions of the queries that will be sent to the recipe chatbot. A dimention describes aspects of the query that that the users send to the chatbot. 
        Given following dimention types, generate tuple combinations of the dimentions

        Dimentions:
            meal_type: The type of meal the user is interested in, such as 'Lite breakfast', 'Brunch'
            dietary_restriction: The dietary restrictions the user has, such as 'Not spicy', 'without fish or meat', 'meaty'
            preparation_time: The preparation time the user is interested in, such as '10 minutes', 'something quick', 'under 1 hour', 'I haveall the time until retirement'

        Instructions:
            - Generate  {num_dimention_instances} tuple combinations of the dimentions.
            - Avoid duplicates. 
            - Make sure that the tuples are diverse. 

        Example output:
        - ('Lite breakfast', 'Not spicy', '10 minutes')
        - ('Snack', 'with hummus and carrots', 'instant')
    """)
    completion = litellm.completion(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": SYSTEM_PROMPT}
        ],
    )
    return completion.choices[0].message.content

In [8]:
dimention_list = generate_dimention_list("")

print(dimention_list)


Here are 50 diverse tuple combinations of the dimensions:

1.  ('Breakfast', 'Not spicy', '10 minutes')
2.  ('Lunch', 'Vegetarian', '30 minutes')
3.  ('Dinner', 'Meaty', 'Under 1 hour')
4.  ('Snack', 'Gluten-free', 'Quick')
5.  ('Dessert', 'Dairy-free', '45 minutes')
6.  ('Brunch', 'Vegan', 'I have all the time until retirement')
7.  ('Appetizer', 'Low-carb', '15 minutes')
8.  ('Side Dish', 'High-protein', '20 minutes')
9.  ('Lite breakfast', 'Kid-friendly', 'Less than 20 minutes')
10. ('Holiday meal', 'Pescatarian', 'All day')
11. ('Party food', 'Sugar-free', 'About an hour')
12. ('Quick bite', 'Nut-free', 'Instant')
13. ('Breakfast', 'Keto', '30 minutes')
14. ('Lunch', 'Heart-healthy', 'Under 1 hour')
15. ('Dinner', 'Spicy', '45 minutes')
16. ('Snack', 'without fish or meat', '10 minutes')
17. ('Dessert', 'Meaty', 'Long prep')
18. ('Brunch', 'Paleo', 'Quick')
19. ('Appetizer', 'Low-salt', '15 minutes')
20. ('Side Dish', 'Diabetic-friendly', '20 minutes')
21. ('Lite breakfast', 'High-

In [29]:
dimention_tuples = [
    ('Breakfast', 'Not spicy', '10 minutes'),
    ('Lunch', 'Vegetarian', '30 minutes'),
    ('Dinner', 'Meaty', 'Under 1 hour'),
    ('Snack', 'Gluten-free', 'Quick'),
    ('Dessert', 'Dairy-free', '45 minutes'),
    ('Brunch', 'Vegan', 'I have all the time until retirement'),
    ('Appetizer', 'Low-carb', '15 minutes'),
    ('Side Dish', 'High-protein', '20 minutes'),
    ('Lite breakfast', 'Kid-friendly', 'Less than 20 minutes'),
    ('Holiday meal', 'Pescatarian', 'All day'),
    ('Party food', 'Sugar-free', 'About an hour'),
    ('Quick bite', 'Nut-free', 'Instant'),
    ('Breakfast', 'Keto', '30 minutes'),
    ('Lunch', 'Heart-healthy', 'Under 1 hour'),
    ('Dinner', 'Spicy', '45 minutes'),
    ('Snack', 'without fish or meat', '10 minutes'),
    ('Dessert', 'Meaty', 'Long prep'),
    ('Brunch', 'Paleo', 'Quick'),
    ('Appetizer', 'Low-salt', '15 minutes'),
    ('Side Dish', 'Diabetic-friendly', '20 minutes'),
    ('Lite breakfast', 'High-fiber', 'Less than 20 minutes'),
    ('Holiday meal', 'Lactose-free', 'Overnight'),
    ('Party food', 'Comfort food', 'About an hour'),
    ('Quick bite', 'with hummus and carrots', 'Instant'),
    ('Breakfast', 'Gluten-free', 'Quick and easy'),
    ('Lunch', 'Vegan', '30 minutes'),
    ('Dinner', 'High-protein', 'Under 1 hour'),
    ('Snack', 'Not spicy', '10 minutes'),
    ('Dessert', 'Vegetarian', '45 minutes'),
    ('Brunch', 'Meaty', 'I have all the time until retirement'),
    ('Appetizer', 'Dairy-free', '15 minutes'),
    ('Side Dish', 'Low-carb', '20 minutes'),
    ('Lite breakfast', 'Pescatarian', 'Less than 20 minutes'),
    ('Holiday meal', 'Kid-friendly', 'All day'),
    ('Party food', 'Nut-free', 'About an hour'),
    ('Quick bite', 'Sugar-free', 'Instant'),
    ('Breakfast', 'Spicy', 'Quick and easy'),
    ('Lunch', 'Keto', '30 minutes'),
    ('Dinner', 'Heart-healthy', 'Under 1 hour'),
    ('Snack', 'Low-salt', '10 minutes'),
    ('Dessert', 'Diabetic-friendly', '45 minutes'),
    ('Brunch', 'High-fiber', 'Long prep'),
    ('Appetizer', 'Lactose-free', '15 minutes'),
    ('Side Dish', 'Comfort food', '20 minutes'),
    ('Lite breakfast', 'Paleo', 'Less than 20 minutes'),
    ('Holiday meal', 'with hummus and carrots', 'Overnight'),
    ('Party food', 'Not spicy', 'About an hour'),
    ('Quick bite', 'Vegetarian', 'Instant'),
    ('Dinner', 'Gluten-free', '10 minutes'),
    ('Lunch', 'Meaty', 'something quick'),
]


In [37]:
from tenacity import retry, wait_fixed, wait_random, before_sleep_log
import logging
import sys

logging.basicConfig(stream=sys.stderr, level=logging.DEBUG)

logger = logging.getLogger(__name__)


@retry(wait=wait_fixed(3) + wait_random(0, 2), before_sleep=before_sleep_log(logger, logging.DEBUG))
def call_llm(dimention: str):
    prompt = f"""
        You are AI assisntant that helps to generate user queryies for a recipe chatbot.
        You will be given a list of tuples, that describe dimentions for a user query. Given these dimentions, generate 4-5 user queries. 

        <dimension>
        {dimention}
        </dimension>

        <example>
        <dimension>
        ('Breakfast', 'Not spicy', '10 minutes')
        </dimension>

        - I want to make a quick breakfast that is not spicy and takes 10 minutes to prepare.
        - Give me a quick recipe for plain breakfast. 
        - Breakfast, fast and non-spicy.
        - A ten-minute breakfast meal, non-spicy.
        - I need breakfast under 10 minutes, it should be mild to taste. 
        </example>

        Just output user queries, no other text.
    """
    completion = litellm.completion(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": prompt}
        ],
    )
    return completion.choices[0].message.content

In [38]:
from collections import defaultdict

tuple_to_queries = defaultdict(list)
for dimention in dimention_tuples[:2]:
    query = call_llm(dimention)
    tuple_to_queries[dimention].append(query)

tuple_to_queries

19:06:22 - LiteLLM:DEBUG: utils.py:340 - 

DEBUG:LiteLLM:

19:06:22 - LiteLLM:DEBUG: utils.py:340 - Request to litellm:
DEBUG:LiteLLM:Request to litellm:
19:06:22 - LiteLLM:DEBUG: utils.py:340 - litellm.completion(model='gemini/gemini-2.5-flash', messages=[{'role': 'user', 'content': "\n        You are AI assisntant that helps to generate user queryies for a recipe chatbot.\n        You will be given a list of tuples, that describe dimentions for a user query. Given these dimentions, generate 4-5 user queries. \n\n        <dimension>\n        ('Breakfast', 'Not spicy', '10 minutes')\n        </dimension>\n\n        <example>\n        <dimension>\n        ('Breakfast', 'Not spicy', '10 minutes')\n        </dimension>\n\n        - I want to make a quick breakfast that is not spicy and takes 10 minutes to prepare.\n        - Give me a quick recipe for plain breakfast. \n        - Breakfast, fast and non-spicy.\n        - A ten-minute breakfast meal, non-spicy.\n        - I need breakfast 

defaultdict(list,
            {('Breakfast',
              'Not spicy',
              '10 minutes'): ["- I'm looking for a quick breakfast recipe that's not spicy and can be made in 10 minutes.\n- Give me a breakfast meal that's mild and takes only 10 minutes to prepare.\n- Need a 10-minute non-spicy breakfast.\n- What's a fast and plain breakfast I can make in under 10 minutes?\n- Breakfast, not spicy, 10 minutes, please."],
             ('Lunch',
              'Vegetarian',
              '30 minutes'): ['- I need a vegetarian lunch that can be made in 30 minutes.\n- Suggest a quick vegetarian lunch recipe ready in half an hour.\n- What are some good vegetarian lunch ideas for a 30-minute prep time?\n- Looking for a fast, meat-free lunch. Must be ready in under 30 minutes.\n- Vegetarian lunch, 30 minutes.']})

In [48]:
from collections import defaultdict

tuple_to_queries = defaultdict(list)
for dimention in dimention_tuples:
    query = call_llm(dimention)
    tuple_to_queries[dimention].append(query)

tuple_to_queries

19:08:35 - LiteLLM:DEBUG: utils.py:340 - 

DEBUG:LiteLLM:

19:08:35 - LiteLLM:DEBUG: utils.py:340 - Request to litellm:
DEBUG:LiteLLM:Request to litellm:
19:08:35 - LiteLLM:DEBUG: utils.py:340 - litellm.completion(model='gemini/gemini-2.5-flash', messages=[{'role': 'user', 'content': "\n        You are AI assisntant that helps to generate user queryies for a recipe chatbot.\n        You will be given a list of tuples, that describe dimentions for a user query. Given these dimentions, generate 4-5 user queries. \n\n        <dimension>\n        ('Breakfast', 'Not spicy', '10 minutes')\n        </dimension>\n\n        <example>\n        <dimension>\n        ('Breakfast', 'Not spicy', '10 minutes')\n        </dimension>\n\n        - I want to make a quick breakfast that is not spicy and takes 10 minutes to prepare.\n        - Give me a quick recipe for plain breakfast. \n        - Breakfast, fast and non-spicy.\n        - A ten-minute breakfast meal, non-spicy.\n        - I need breakfast 


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



19:10:15 - LiteLLM:DEBUG: utils.py:340 - 

DEBUG:LiteLLM:

19:10:15 - LiteLLM:DEBUG: utils.py:340 - Request to litellm:
DEBUG:LiteLLM:Request to litellm:
19:10:15 - LiteLLM:DEBUG: utils.py:340 - litellm.completion(model='gemini/gemini-2.5-flash', messages=[{'role': 'user', 'content': "\n        You are AI assisntant that helps to generate user queryies for a recipe chatbot.\n        You will be given a list of tuples, that describe dimentions for a user query. Given these dimentions, generate 4-5 user queries. \n\n        <dimension>\n        ('Holiday meal', 'Lactose-free', 'Overnight')\n        </dimension>\n\n        <example>\n        <dimension>\n        ('Breakfast', 'Not spicy', '10 minutes')\n        </dimension>\n\n        - I want to make a quick breakfast that is not spicy and takes 10 minutes to prepare.\n        - Give me a quick recipe for plain breakfast. \n        - Breakfast, fast and non-spicy.\n        - A ten-minute breakfast meal, non-spicy.\n        - I need break


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



19:10:20 - LiteLLM:DEBUG: utils.py:340 - 

DEBUG:LiteLLM:

19:10:20 - LiteLLM:DEBUG: utils.py:340 - Request to litellm:
DEBUG:LiteLLM:Request to litellm:
19:10:20 - LiteLLM:DEBUG: utils.py:340 - litellm.completion(model='gemini/gemini-2.5-flash', messages=[{'role': 'user', 'content': "\n        You are AI assisntant that helps to generate user queryies for a recipe chatbot.\n        You will be given a list of tuples, that describe dimentions for a user query. Given these dimentions, generate 4-5 user queries. \n\n        <dimension>\n        ('Holiday meal', 'Lactose-free', 'Overnight')\n        </dimension>\n\n        <example>\n        <dimension>\n        ('Breakfast', 'Not spicy', '10 minutes')\n        </dimension>\n\n        - I want to make a quick breakfast that is not spicy and takes 10 minutes to prepare.\n        - Give me a quick recipe for plain breakfast. \n        - Breakfast, fast and non-spicy.\n        - A ten-minute breakfast meal, non-spicy.\n        - I need break

...


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



19:12:26 - LiteLLM:DEBUG: utils.py:340 - 

DEBUG:LiteLLM:

19:12:26 - LiteLLM:DEBUG: utils.py:340 - Request to litellm:
DEBUG:LiteLLM:Request to litellm:
19:12:26 - LiteLLM:DEBUG: utils.py:340 - litellm.completion(model='gemini/gemini-2.5-flash', messages=[{'role': 'user', 'content': "\n        You are AI assisntant that helps to generate user queryies for a recipe chatbot.\n        You will be given a list of tuples, that describe dimentions for a user query. Given these dimentions, generate 4-5 user queries. \n\n        <dimension>\n        ('Side Dish', 'Comfort food', '20 minutes')\n        </dimension>\n\n        <example>\n        <dimension>\n        ('Breakfast', 'Not spicy', '10 minutes')\n        </dimension>\n\n        - I want to make a quick breakfast that is not spicy and takes 10 minutes to prepare.\n        - Give me a quick recipe for plain breakfast. \n        - Breakfast, fast and non-spicy.\n        - A ten-minute breakfast meal, non-spicy.\n        - I need breakfa


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



19:12:30 - LiteLLM:DEBUG: utils.py:340 - 

DEBUG:LiteLLM:

19:12:30 - LiteLLM:DEBUG: utils.py:340 - Request to litellm:
DEBUG:LiteLLM:Request to litellm:
19:12:30 - LiteLLM:DEBUG: utils.py:340 - litellm.completion(model='gemini/gemini-2.5-flash', messages=[{'role': 'user', 'content': "\n        You are AI assisntant that helps to generate user queryies for a recipe chatbot.\n        You will be given a list of tuples, that describe dimentions for a user query. Given these dimentions, generate 4-5 user queries. \n\n        <dimension>\n        ('Side Dish', 'Comfort food', '20 minutes')\n        </dimension>\n\n        <example>\n        <dimension>\n        ('Breakfast', 'Not spicy', '10 minutes')\n        </dimension>\n\n        - I want to make a quick breakfast that is not spicy and takes 10 minutes to prepare.\n        - Give me a quick recipe for plain breakfast. \n        - Breakfast, fast and non-spicy.\n        - A ten-minute breakfast meal, non-spicy.\n        - I need breakfa

defaultdict(list,
            {('Breakfast',
              'Not spicy',
              '10 minutes'): ["- I'm looking for a sweet vegetarian dinner recipe that takes about 30 minutes.\n- Can you suggest a quick vegetarian dinner? Something sweet and ready in half an hour.\n- Dinner, vegetarian, sweet, 30 min.\n- What's a good sweet vegetarian dinner I can make in under 30 minutes?\n- I need a 30-minute vegetarian dinner. Make it sweet."],
             ('Lunch',
              'Vegetarian',
              '30 minutes'): ["- I need a vegetarian lunch recipe that I can make in 30 minutes.\n- Give me ideas for a quick vegetarian lunch, ready in half an hour.\n- What's a good vegetarian lunch that takes 30 minutes or less?\n- Looking for a fast, vegetarian lunch recipe, 30-minute prep time.\n- Show me some 30-minute vegetarian lunch options."],
             ('Dinner',
              'Meaty',
              'Under 1 hour'): ["- I'm looking for a meaty dinner recipe that takes less than an hour to

In [49]:
len(tuple_to_queries)

50

In [2]:
for dimention, queries in tuple_to_queries.items():
    print(f"""\n===={" + ".join(dimention)}====""")
    for query in queries:
        print(query)


NameError: name 'tuple_to_queries' is not defined

In [11]:
from pathlib import Path
current_dir = Path.cwd()
print(current_dir)

synthetic_queries = []
synthetic_queries_txt = (current_dir / "synthetic_queries.txt").read_text()
for idx, query in enumerate(synthetic_queries_txt.split("\n")):
    if (line := query.strip()).startswith("-"):
        line = line.lstrip("-").removeprefix('"').removeprefix("'").removesuffix('"').strip()
        synthetic_queries.append({"id": idx, "query": line})
import pandas as pd
df = pd.DataFrame(synthetic_queries)
df.to_csv("synthetic_queries.csv", index=False)

/Users/home/Projects/recipe-chatbot/homeworks/hw2
